In [1]:
import os
from os_snippets import remove_empty_folders
import numpy as np
import pandas as pd
import cv2
from scipy.io import loadmat
from skimage.io import imread
from skimage.io import imsave
from glob import glob
from tqdm import tqdm
import matlab.engine
from all_in_focus import all_in_focus
from focal_stack_legacy import focal_stack
from dft_registration import register_with_ref
from dft_registration import register_pair
from MIST_run_ijm import create_mist_command
from MIST_run_ijm import run_ijm
from stitch_meta import stitch_from_meta
from cell_rna_report import generate_report
from report_xetex import integrate_report

#base_directory = r'D:\FISH_images'
base_directory = r'\\192.168.12.12\home\FISH_DATA_PKU_Archive'
base_dest_directory = r'E:\FISH_images_processed'
RUN_ID = '20210614_brain_10plex_001'
raw_directory = os.path.join(base_directory,RUN_ID)
dest_directory = os.path.join(base_dest_directory,f'{RUN_ID}_processed')
aif_directory = os.path.join(dest_directory,'focal_stacked')
sdc_directory = os.path.join(dest_directory,'background_corrected')
rgs_directory = os.path.join(dest_directory,'registered')
stc_directory = os.path.join(dest_directory,'stitched')
report_directory = os.path.join(dest_directory,'report')

def try_mkdir(d):
    try:
        os.makedirs(d)
    except FileExistsError:
        print(f'{d} exists.')
        pass

try_mkdir(dest_directory)
try_mkdir(aif_directory)
try_mkdir(stc_directory)
try_mkdir(rgs_directory)
try_mkdir(report_directory)

ref_cycle = 1
ref_channel = 'cy3'
nuclei_cyc_chn = 'cyc_10_DAPI'
report_cyc_chns = ['cyc_2_cy3','cyc_2_cy5']

even_less = False
cyc_zero = False

#channels = ['cy3','cy5','FAM','DAPI']
channels = ['cy3','cy5']

E:\FISH_images_processed\20210614_brain_10plex_001_processed exists.
E:\FISH_images_processed\20210614_brain_10plex_001_processed\focal_stacked exists.
E:\FISH_images_processed\20210614_brain_10plex_001_processed\stitched exists.
E:\FISH_images_processed\20210614_brain_10plex_001_processed\registered exists.
E:\FISH_images_processed\20210614_brain_10plex_001_processed\report exists.


## All-in-focus Image Stacking

In [2]:
def imlist_to_df(imlist):
    imlist_2d = [s.strip('.tif').split('-') for s in imlist]
    df = pd.DataFrame(imlist_2d, columns=['Cycle','Tile','Channel','Z'])
    df = df.loc[:,['Cycle','Tile','Z']]
    for column in ['Cycle','Tile','Z']:
        df[column] = df[column].apply(lambda x: int(x[1:]))
    return df

try:
    tile_info = loadmat(os.path.join(raw_directory,'TileInfo.mat'))
except FileNotFoundError as exc:
    raise FileNotFoundError('Tile info file not included') from exc
tile_x = int(tile_info['TileX'])
tile_y = int(tile_info['TileY'])
tile_num = tile_x * tile_y
tiles = [i+1 for i in range(tile_num)]
cycles = glob(os.path.join(raw_directory,'cyc_*')) if not even_less else glob(os.path.join(raw_directory,'cyc_*[13579]'))
cycles += glob(os.path.join(raw_directory,'cyc_0')) if cyc_zero and even_less else []

def get_stack_num(path):
    sample = [f for f in os.listdir(path) if f.endswith('.tif')][0]
    prefix = sample.split('-Z')[0]
    stack_num = max([int(f.strip('.tif').split('-Z')[1]) for f in os.listdir(path) if f.startswith(prefix)]) + 1
    return stack_num

stack_num = get_stack_num(cycles[0])
channel_info = {}
for channel in channels:
    channel_info[channel] = imlist_to_df([f for f in os.listdir(cycles[0]) if channel in f])
    if glob(os.path.join(raw_directory,'cyc_*',f'*-{channel}-*')):
        eng = matlab.engine.start_matlab()
        eng.cd('C:/Users/Dell/Documents/LabView_FISH_PKU/FISH_analysis_pipeline/fstack')
        eng.fstack_stacknum_channel(raw_directory,aif_directory,stack_num,channel,nargout=0)
        eng.quit()
remove_empty_folders(aif_directory)    

## Background Shading Correction

In [3]:
try_mkdir(sdc_directory)
eng = matlab.engine.start_matlab()
eng.cd('C:/Users/Dell/Documents/LabView_FISH_PKU/FISH_analysis_pipeline/CIDRE')
eng.background_correction(aif_directory,sdc_directory,nargout=0)
eng.quit()

## Registration

In [4]:
try:
    captured_tiles = list(channel_info[ref_channel]['Tile'].unique())
    ref_channel_alt = ref_channel
except KeyError:
    for key in channel_info:
        captured_tiles = list(channel_info[key]['Tile'].unique())
        ref_channel_alt = key
        break
img_names = [f'FocalStack_{t:03d}.tif' for t in captured_tiles]
register_with_ref(sdc_directory,rgs_directory,channels,img_names,ref_cycle=ref_cycle,ref_channel=ref_channel_alt)


100%|██████████| 50/50 [10:22<00:00, 12.44s/it]


## Image Stitching

In [6]:
registered_cyc_chn = os.listdir(rgs_directory)
empty_im = np.zeros((2048,2048),dtype=np.uint16)
for cyc_chn in registered_cyc_chn:
    imgs = [f for f in os.listdir(os.path.join(rgs_directory,cyc_chn)) if f.endswith('.tif')]
    for tile in tiles:
        if f'FocalStack_{tile:03d}.tif' not in imgs:
            imsave(os.path.join(rgs_directory,cyc_chn,f'FocalStack_{tile:03d}.tif'),empty_im)

In [4]:
stitch_template = f'cyc_{ref_cycle}_{ref_channel}'
create_mist_command(tile_x,tile_y,os.path.join(rgs_directory,stitch_template).replace('\\',r'\\\\'),stc_directory.replace('\\',r'\\\\'))
if glob(os.path.join(stc_directory,f'*global*')):
    pass
else:
    run_ijm('MIST_temp.ijm')
    # os.rename(os.path.join(stc_directory,'img-stitched-0.tif'),os.path.join(stc_directory,f'{stitch_template}.tif'))
stitch_from_meta(stc_directory,rgs_directory,stitch_template)

100%|██████████| 15/15 [04:19<00:00, 17.32s/it]


## Cell Segmentation

In [7]:
if not glob(os.path.join(dest_directory,'CellYX.mat')):
    eng = matlab.engine.start_matlab()
    eng.cd('C:/Users/Dell/Documents/LabView_FISH_PKU/FISH_analysis_pipeline')
    eng.cell_segmentation(os.path.join(stc_directory,f'{nuclei_cyc_chn}.tif'),dest_directory,nargout=0)
    eng.quit()
else:
    print('Already segmented.')

MatlabExecutionError: 
  File C:\Program Files\MATLAB\R2020b\toolbox\matlab\imagesci\imread.m, line 570, in get_full_filename

  File C:\Program Files\MATLAB\R2020b\toolbox\matlab\imagesci\imread.m, line 377, in imread

  File C:\Users\Dell\Documents\LabView_FISH_PKU\FISH_analysis_pipeline\cell_segmentation.m, line 3, in cell_segmentation
File "E:\FISH_images_processed\20210902-dNTPtest-3_processed\stitched\cyc_10_DAPI.tif" does not exist.


In [ ]:
im = imread(os.path.join(stc_directory,'cyc_1_cy3.tif'))
canvas = np.zeros(im.shape,dtype=np.uint16)
for i in range(1,11):
    for chn in ['cy3','cy5']:
        img = imread(os.path.join(stc_directory,f'cyc_{i}_{chn}.tif'))
        canvas += img / 20
canvas = cv2.GaussianBlur(canvas,(9,9))
imsave(os.path.join(stc_directory,'mean_cycle.tif'),canvas)

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('uint16') with casting rule 'same_kind'

In [ ]:
eng = matlab.engine.start_matlab()
eng.cd('C:/Users/Dell/Documents/LabView_FISH_PKU/FISH_analysis_pipeline')
eng.cell_segmentation(os.path.join(stc_directory,f'{nuclei_cyc_chn}.tif'),dest_directory,nargout=0)
eng.quit()

## Generate Report

In [ ]:
for cyc_chn in report_cyc_chns:
    cyc,chn = cyc_chn.split('_')[1:]
    generate_report(RUN_ID,cyc,chn, 4 if chn=='cy5' else 2)
integrate_report(RUN_ID,report_directory)
os.chdir(r'C:\Users\Dell\Documents\LabView_FISH_PKU\FISH_analysis_pipeline')